In [ ]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Linq;
using System.Net.Http;
using System.Text;
using System.Threading.Tasks;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
#r "nuget:RestSharp"
using RestSharp;

In [ ]:
public class InspectItem
{
    public string? id { get; set; }
    public string?  imageName { get; set; }
    public string?  productId { get; set; }
    public string?  layerId { get; set; }
    public string?  equipmentId{ get; set; }
    public string?  direction{ get; set; }
    public int  status{ get; set; }
    public string? defectBoxs{ get; set; }
    public string?  note{ get; set; }
    public DateTime?  startTime{ get; set; }
    public DateTime? endTime{ get; set; }
    public string?  imagePath{ get; set; }
    public string?  ftpPath{ get; set; }
    public string?  extendOne{ get; set; }
    public string?  extendTwo{ get; set; }
    public string?  extendThree{ get; set; }
    public string?  createBy{ get; set; }
    public DateTime? createDate{ get; set; }
    public string? updateBy{ get; set; }
    public DateTime? updateDate{ get; set; }
}

In [ ]:
public class GroupData
{
    public string EqpName;
    public List<InspectItem> InspectItems;
    public int finishedItem { get 
        {
            int count = 0;
            foreach (var item in InspectItems)
            {
                if (item.status == 1)
                {
                    count++;
                }
            }
            return count;
        } }
    public int totalItems { get { return InspectItems.Count; } }

    public GroupData(string EqpName,List<InspectItem> items)
    {
        this.EqpName = EqpName;
        this.InspectItems = new List<InspectItem>();
    }
}

In [ ]:
public static class ServerConnector
{
    static RestClient restClient = new RestClient("http://10.141.34.78:26208/EAC/");

    static ServerConnector()
    {
        restClient.Options.MaxTimeout = 10000;
    }
    public static InspectItem[] GetInfo(DateTime starttime,DateTime endtime)
    {            
        var request = new RestRequest("getImageInfo");

        request.AddQueryParameter("startTime", starttime.ToString("yyyy-MM-dd HH:mm:ss"));
        request.AddQueryParameter("endTime", endtime.ToString("yyyy-MM-dd HH:mm:ss"));

        var response = restClient.Get(request);
        if (response.IsSuccessful)
        {
            if (response.Content == null)
            {
                throw new Exception("获取的产品信息为空；");
            }
            else
            {
                string jsonstring = response.Content;
                InspectItem[] deserializedProduct = JsonConvert.DeserializeObject<InspectItem[]>(jsonstring);
                return deserializedProduct;
            }
        }
        else
        {
            throw new Exception("连接失败；");
        }
    }
    public static List<GroupData> GetGroupedData(InspectItem[] data)
    {
        List<GroupData> groupedData = new List<GroupData>();
        var eqplist = from item in data
                        group item.equipmentId by item.equipmentId into g
                        select new { equipmentId = g.Key, Count = g.Count() };

        foreach (var eq in eqplist)
        {
            var items = from item in data where item.equipmentId == eq.equipmentId select item;

            var newgroup = new GroupData(eq.equipmentId, items.ToList());
            groupedData.Add(newgroup);
        }
        return groupedData;
    }
    public static void SendResult(string id,int status)
    {
        var request = new RestRequest("modifyImageStatus",Method.Post);
        var jsonresult = new JObject();
        jsonresult["id"] = id;
        jsonresult["status"] = status;

        request.AddJsonBody(jsonresult.ToString());
        var response = restClient.Post(request);
        if (response.IsSuccessful)
        {
        }
        else
        {
            throw new Exception("上传检查结果失败，请检查服务器连接；");
        }
    }
    public static Stream GetImage(string id)
    {
        var request = new RestRequest("getImage");

        request.AddQueryParameter("imageId", id);

        var response = restClient.Get(request);
        if (response.IsSuccessful)
        {
            if (response.RawBytes == null)
            {
                throw new Exception("获取的图像为空；");
            }
            else
            {
            return new MemoryStream(response.RawBytes);
            }
        }
        else
        {
            throw new Exception("连接失败；");
        }
    }
}

In [ ]:
DateTime start = DateTime.Parse("2022-06-17 00:10:00");
DateTime end = DateTime.Parse("2022-06-18 00:10:00");
var data = ServerConnector.GetInfo(start,end);
data

In [ ]:
var listdata = ServerConnector.GetGroupedData(data);
listdata

In [ ]:
string imgid = "699abb4363404f869b4ff5c9ecb0467e";
var imgdata = ServerConnector.GetImage(imgid);

In [ ]:
string path = @"D:\Program\Mordor2.0\CutInspect\test.json";
FileInfo aaa = new FileInfo(path);
var data = aaa.OpenRead();
string jsonstring = aaa.OpenText().ReadToEnd();
jsonstring

In [ ]:
InspectItem[] deserializedProduct = JsonConvert.DeserializeObject<InspectItem[]>(jsonstring);
deserializedProduct